# Semi-structured RAG

## Load environment variables

In [1]:
from dotenv import load_dotenv

load_dotenv('app/src/shared/.env')

True

## Load data 

As normas brasileiras relacionadas a instalações elétricas abrangem uma variedade de aspectos, desde baixa tensão até instalações específicas em locais como áreas classificadas e sistemas fotovoltaicos. As principais normas da Associação Brasileira de Normas Técnicas (ABNT) sobre instalações elétricas incluem:

* NBR 5410 - Instalações elétricas de baixa tensão: Trata das condições para o projeto, execução e manutenção de instalações elétricas de baixa tensão.

* NBR 14039 - Instalações elétricas de média tensão de 1,0 kV a 36,2 kV: Estabelece as condições para projeto, execução e manutenção dessas instalações.

* NBR 5413 - Iluminância de interiores: Define os requisitos para níveis de iluminância em ambientes internos.

* NBR 13570 -  Instalações Elétricas em Locais de Afluência de Público - Requisitos específicos

* ABNT NBR IEC 60079-14 - Instalações elétricas em atmosferas explosivas - Área classificada: Requisitos para instalações em áreas com risco de explosão.

* NBR 10898 - Sistemas de iluminação de emergência: Especifica os requisitos para sistemas de iluminação de emergência em edifícios.

* NBR 15514 - Recipientes transportáveis de gás liquefeito de petróleo (GLP) — Área de armazenamento — Requisitos de segurança
    * Embora a norma NBR 15514 não seja diretamente uma norma de instalações elétricas, ela possui interseções com a área elétrica em aspectos relacionados à segurança, especialmente devido aos riscos potenciais de explosões e incêndios associados ao GLP.
* NBR 5419 - Proteção contra descargas atmosféricas (em quatro partes):

    * Parte 1: Princípios gerais
    * Parte 2: Gerenciamento de risco
    * Parte 3: Danos físicos a edificações e perigos à vida
    * Parte 4: Sistemas elétricos e eletrônicos internos na estrutura

    * NBR 16280 - Reforma em edificações - Sistema de gestão de reformas: Estabelece requisitos para reformas em edificações, incluindo instalações elétricas.


Essas são algumas das principais normas que tratam de diversos aspectos das instalações elétricas no Brasil, garantindo segurança, eficiência e conformidade técnica.

In [2]:
import os
import logging
from concurrent.futures import ThreadPoolExecutor
from typing import List
from unstructured.partition.pdf import partition_pdf

# Configure logging 
logging.basicConfig(filename='pdf_processing.log', level=logging.INFO, format='%(asctime)s %(message)s')

def process_pdf_file(filename, path):
    """Processes a single PDF file and returns the extracted elements.

    Logs information and errors during processing.
    """
    try:
        raw_pdf_elements = partition_pdf(
            filename=os.path.join(path, filename),  # Combine path and filename
            extract_images_in_pdf=False,
            infer_table_structure=True,
            chunking_strategy="by_title",
            max_characters=4000,
            new_after_n_chars=3800,
            combine_text_under_n_chars=2000
        )
        logging.info(f"Successfully processed PDF: {filename}")
        return raw_pdf_elements
    except Exception as e:  # Catch specific PDF processing errors
        logging.error(f"Error processing PDF: {filename} - {e}")
        return []

def process_single_pdf(pdf_folder_path: str, filename: str):
    """Helper function to process a single PDF file."""
    if filename.endswith(".pdf"):
        logging.info(f"Reading PDF doc: {filename}")
        return process_pdf_file(filename, pdf_folder_path)
    return []

# Loop through PDF Files with ThreadPoolExecutor:
def process_multiple_pdfs(pdf_folder_path: str, max_workers: int = 4) -> List:
    """Processes all PDF files within a specified folder using ThreadPoolExecutor.

    Logs information and errors during processing.
    """
    all_elements = []
    filenames = [filename for filename in os.listdir(pdf_folder_path) if filename.endswith(".pdf")]

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = executor.map(lambda filename: process_single_pdf(pdf_folder_path, filename), filenames)

    for result in results:
        all_elements.extend(result)  # Extend the all_elements list

    return all_elements

In [3]:
pdf_folder_path = "/Users/nathan/workspace/tcc/test"
raw_pdfs_elements = process_multiple_pdfs(pdf_folder_path)

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
len(raw_pdfs_elements)

17

In [5]:
raw_pdfs_elements

In [6]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in raw_pdfs_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 15,
 "<class 'unstructured.documents.elements.Table'>": 2}

In [7]:
from typing import Any
from pydantic import BaseModel

# Define the Element class based on potential types returned by partition_pdf
class Element(BaseModel):
    type: str  # Textual representation of the element type (e.g., "table", "text")
    text: Any  # Content of the element, can be text, tables, or other data structures

# Categorize by type
def categorize_elements(raw_pdf_elements) -> list[Element]:
    """Categorizes elements by type and returns a dictionary with counts."""
    categorized_elements = [] # Initialize category counts
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Table" in str(type(element)):
            categorized_elements.append(Element(type="table", text=str(element)))
        elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
            categorized_elements.append(Element(type="text", text=str(element)))
    return categorized_elements

In [8]:
# Improved categorization function with clear structure
all_categories = categorize_elements(raw_pdfs_elements)
print(len(all_categories))

# Tables
table_elements = [e for e in all_categories if e.type == "table"]
print(f"Number of tables: {len(table_elements)}")

# Text
text_elements = [e for e in all_categories if e.type == "text"]
print(f"Number of text elements: {len(text_elements)}")

17
Number of tables: 2
Number of text elements: 15


In [9]:
text_elements[10]

Element(type='text', text='10.13 - RESPONSABILIDADES\n\n10.13.1 As responsabilidades quanto ao cumprimento desta NR são solidárias aos contratantes e contratados envolvidos. (Revogado pela Portaria SEPRT n.º 915, de 30 de julho de 2019)\n\n10.13.2 É de responsabilidade dos contratantes manter os trabalhadores informados sobre os riscos a que estão expostos, instruindo-os quanto aos procedimentos e medidas de controle contra os riscos elétricos a serem adotados.\n\n10.13.3 Cabe à empresa, na ocorrência de acidentes de trabalho envolvendo instalações e serviços em eletricidade, propor e adotar medidas preventivas e corretivas.\n\n10.13.4 Cabe aos trabalhadores:\n\na) zelar pela sua segurança e saúde e a de outras pessoas que possam ser afetadas por suas ações ou omissões no trabalho;\n\nb) responsabilizar-se junto com a empresa pelo cumprimento das disposições legais e regulamentares, inclusive quanto aos procedimentos internos de segurança e saúde; e\n\nc) comunicar, de imediato, ao res

In [10]:
table_elements

[Element(type='table', text='Publicação D.O.U. 06/07/78 Alterações/Atualizações D.O.U. 14/06/83 08/09/04 02/05/16 31/07/19'),
 Element(type='table', text='Rr - Raio de Faixa delimitação tensão entre zona Nominal da de risco e instalação controlada elétrica em em metros kV 0,20 1 0,22 1 e 3 0,25 3 e 6 0,35 6 e 10 0,38 10 e 15 0,40 15 e 20 0,56 20 e 30 0,58 30 e 36 0,63 36 e 45 0,83 45 e 60 0,90 60 e 70 1,00 70 e 110 110 e 132 1,10 132 e 150 1,20 150 e 220 1,60 220 e 275 1,80 275 e 380 2,50 380 e 480 3,20 480 e 700 5,20 Rc - Raio de delimitação entre zona controlada e livre em metros 0,70 1,22 1,25 1,35 1,38 1,40 1,56 1,58 1,63 1,83 1,90 2,00 3,10 3,20 3,60 3,80 4,50 5,20 7,20')]

### Data analytics

In [ ]:
# analisar os crunks pai com filho(summarized by gemini model)
# import pandas as pd

# text_data = {"Texts": texts, "Text_summaries": text_summaries}
# df = pd.DataFrame(text_data)

# df.head()

## Add to vectorstore
Use Multi Vector Retriever with summaries:

- InMemoryStore stores the raw text, tables
- vectorstore stores the embedded summaries

In [11]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", task_type="retrieval_document")
gemini_embeddings

GoogleGenerativeAIEmbeddings(model='models/embedding-001', task_type='retrieval_document', google_api_key=None, credentials=None, client_options=None, transport=None)

#### Creating the Vectorial Database

In [12]:
import sqlite3
import pandas as pd
from langchain_community.vectorstores import Chroma
persist_directory = "./chroma_db"

# setup to save to disk
vectorstore = Chroma(collection_name="NR10", 
                        embedding_function=gemini_embeddings,
                        persist_directory=persist_directory)

# Conexão ao banco de dados SQLite
conn = sqlite3.connect('chroma_db/chroma.sqlite3')

tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)
print(tables, "\n")

# Executa a consulta e retorna um DataFrame
df = pd.read_sql_query("SELECT * FROM embedding_fulltext_search", conn)

# Exibe o DataFrame
print(df,"\n\n")

conn.close()

                                 name
0                          migrations
1                    embeddings_queue
2                 collection_metadata
3                            segments
4                    segment_metadata
5                             tenants
6                           databases
7                         collections
8                          embeddings
9                  embedding_metadata
10                         max_seq_id
11          embedding_fulltext_search
12     embedding_fulltext_search_data
13      embedding_fulltext_search_idx
14  embedding_fulltext_search_content
15  embedding_fulltext_search_docsize
16   embedding_fulltext_search_config 

Empty DataFrame
Columns: [string_value]
Index: [] 




#### Add the documents to the Vectorial Database

In [13]:
import uuid
from langchain_core.documents import Document

# Apply to tables
tables = [i.text for i in table_elements]
# Apply to texts
texts = [i.text for i in text_elements]

# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
id_key = "doc_id"

page_content_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(texts)
]

page_content_tables = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(tables)
]

vectorstore.add_documents(page_content_texts)
vectorstore.add_documents(page_content_tables)

['08a857f5-0917-44e2-8616-9ac6942caafd',
 '93f2716f-35ce-407c-916b-6682a2fbf464']

In [14]:
import sqlite3
import pandas as pd

# Conexão ao banco de dados SQLite
conn = sqlite3.connect('chroma_db/chroma.sqlite3')

# Executa a consulta e retorna um DataFrame
df = pd.read_sql_query("SELECT * FROM embedding_fulltext_search", conn)

# Exibe o DataFrame
print(df,"\n\n")
print(f"Colunas: {df.columns} \n")

# print(df['string_value'].iloc[1])

# Fecha a conexão
conn.close()

                                         string_value
0   NR 10 - SEGURANÇA EM INSTALAÇÕES E SERVIÇOS EM...
1   Portaria MTb n.º 3.214, de 08 de junho de 1978...
2   10.2 - MEDIDAS DE CONTROLE\n\n10.2.1 Em todas ...
3   10.2.9.1 Nos trabalhos em instalações elétrica...
4   segurança:\n\na) especificação das característ...
5   f) instalação da sinalização de impedimento de...
6   10.7 - TRABALHOS ENVOLVENDO ALTA TENSÃO (AT)\n...
7   10.8.5 A empresa deve estabelecer sistema de i...
8   10.9 - PROTEÇÃO CONTRA INCÊNDIO E EXPLOSÃO\n\n...
9   10.11 - PROCEDIMENTOS DE TRABALHO\n\n10.11.1 O...
10  10.13 - RESPONSABILIDADES\n\n10.13.1 As respon...
11  GLOSSÁRIO\n\n1. Alta Tensão (AT): tensão super...
12  25. Sistema Elétrico: circuito ou circuitos el...
13  Figura 1 - Distâncias no ar que delimitam radi...
14  a) noções básicas; b) medidas preventivas; c) ...
15  Publicação D.O.U. 06/07/78 Alterações/Atualiza...
16  Rr - Raio de Faixa delimitação tensão entre zo... 


Colunas: Index(['string_v

### Visualization of the vector DB

In [ ]:
import sqlite3
import pandas as pd

# Conexão ao banco de dados SQLite
conn = sqlite3.connect('chroma_db/chroma.sqlite3')

tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)

print(tables)

conn.close()

In [ ]:
import sqlite3
import pandas as pd

# Conexão ao banco de dados SQLite
conn = sqlite3.connect('chroma_db/chroma.sqlite3')

# Consulta SQL
query = "SELECT * FROM embedding_fulltext_search"

# Executa a consulta e retorna um DataFrame
df = pd.read_sql_query(query, conn)

# Exibe o DataFrame
print(df,"\n\n")
print(f"Colunas: {df.columns}")
print(df['string_value'].iloc[1])

# Fecha a conexão
conn.close()

## RAG Pipeline

### Retriver

#### Vector store-backed retriever

In [42]:
# Load from disk
# vectorstore_db = Chroma(
#                         persist_directory="./chroma_db",       # Directory of db
#                         embedding_function=gemini_embeddings   # Embedding model
#                    )

# retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

# retriever = vectorstore.as_retriever(
#     search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.7}
# )

retriever = vectorstore.as_retriever(search_type="mmr")

retriever

VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x283047260>, search_type='mmr')

In [ ]:
query = "O que diz o topico 10.2.8.1 da norma regulamentadora NR 10?"
docs = retriever.invoke(query)
docs

#### Parent Document Retriever

#### Multivector Retriver

In [ ]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

persist_directory = "./chroma_db"

# Load from disk
vectorstore = Chroma(collection_name="summaries", 
                        embedding_function=gemini_embeddings,
                        persist_directory=persist_directory)

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableMap

model = GoogleGenerativeAI(model="gemini-pro", temperature=0.3, top_p=0.85)

# Prompt template
template = """Você é um assistente chamado Spark e sua função é responder dúvidas e questionamentos relacionadas as instalações elétricas brasileiras com base no contexto, o qual pode incluir textos e/ou tabelas referentes as normas brasileiras (NBRs):
{context}
Question: {question}

"""
prompt = ChatPromptTemplate.from_template(template)

# Função para imprimir o contexto
def print_context(context):
    print("Contexto fornecido:", context)
    return context

# RAG pipeline com etapa intermediária para capturar e imprimir o contexto
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | RunnableMap({"context": print_context, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
# Invocar a chain e imprimir o resultado
result = chain.invoke("O que diz o topico 10.2.8.1 da norma regulamentadora NR 10?")
print("Resposta:", result)

### Generate

In [48]:
import textwrap
from IPython.display import Markdown
from langchain import PromptTemplate

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Prompt template to query Gemini
llm_prompt_template = """Você é um assistente para tarefas de resposta a perguntas.
Use o contexto como base para responder à pergunta.
Dê uma resposta da maneira mais completa e ao mesmo tempo objetiva.\n
Pergunta: {question} \nContext: {context} \nResposta:"""

llm_prompt = PromptTemplate.from_template(llm_prompt_template)


from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(model="gemini-pro",
                 temperature=0.3, top_p=0.85)


from langchain_core.runnables import RunnableMap
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser

# Combine data from documents to readable string format.
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Função para imprimir o contexto
def print_context(context):
    print("Contexto fornecido:", context)
    return context

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | RunnableMap({"context": print_context, "question": RunnablePassthrough()})
    | llm_prompt
    | llm
    | StrOutputParser()
)

In [17]:
query = "Qual é o objetivo principal da NR 10?"
rag_chain.invoke(query)

Contexto fornecido: {'context': 'NR 10 - SEGURANÇA EM INSTALAÇÕES E SERVIÇOS EM ELETRICIDADE', 'question': 'Qual é o objetivo principal da NR 10?'}


'Estabelecer requisitos e medidas de proteção para garantir a segurança e a saúde dos trabalhadores em instalações e serviços em eletricidade.'

In [26]:
query = "Em quais tipos de atividades a NR 10 deve ser aplicada?"
to_markdown(rag_chain.invoke(query))

Contexto fornecido: {'context': 'NR 10 - SEGURANÇA EM INSTALAÇÕES E SERVIÇOS EM ELETRICIDADE', 'question': 'Em quais tipos de atividades a NR 10 deve ser aplicada?'}


> A NR 10 deve ser aplicada em todas as atividades que envolvam geração, transmissão, distribuição e utilização de energia elétrica, incluindo:
> 
> * Instalação, manutenção e operação de linhas de transmissão e distribuição
> * Instalação, manutenção e operação de subestações
> * Instalação, manutenção e operação de usinas geradoras
> * Instalação, manutenção e operação de equipamentos elétricos em indústrias, comércios e residências
> * Serviços de eletricidade, como inspeções, testes e reparos

In [38]:
query = "Quais são as responsabilidades dos empregadores e dos trabalhadores em relação à NR 10?"
to_markdown(rag_chain.invoke(query))

Contexto fornecido: {'context': '10.13 - RESPONSABILIDADES\n\n10.13.1 As responsabilidades quanto ao cumprimento desta NR são solidárias aos contratantes e contratados envolvidos. (Revogado pela Portaria SEPRT n.º 915, de 30 de julho de 2019)\n\n10.13.2 É de responsabilidade dos contratantes manter os trabalhadores informados sobre os riscos a que estão expostos, instruindo-os quanto aos procedimentos e medidas de controle contra os riscos elétricos a serem adotados.\n\n10.13.3 Cabe à empresa, na ocorrência de acidentes de trabalho envolvendo instalações e serviços em eletricidade, propor e adotar medidas preventivas e corretivas.\n\n10.13.4 Cabe aos trabalhadores:\n\na) zelar pela sua segurança e saúde e a de outras pessoas que possam ser afetadas por suas ações ou omissões no trabalho;\n\nb) responsabilizar-se junto com a empresa pelo cumprimento das disposições legais e regulamentares, inclusive quanto aos procedimentos internos de segurança e saúde; e\n\nc) comunicar, de imediato, 

> **Responsabilidades dos Empregadores:**
> 
> * Manter os trabalhadores informados sobre os riscos elétricos.
> * Instruir os trabalhadores sobre os procedimentos e medidas de controle contra riscos elétricos.
> * Propor e adotar medidas preventivas e corretivas em caso de acidentes de trabalho envolvendo eletricidade.
> * Promover ações de controle de riscos originados por terceiros em suas instalações elétricas.
> * Denunciar imediatamente aos órgãos competentes qualquer descumprimento das normas da NR 10.
> 
> **Responsabilidades dos Trabalhadores:**
> 
> * Zelar pela própria segurança e saúde, bem como pela segurança e saúde de outras pessoas que possam ser afetadas por suas ações ou omissões no trabalho.
> * Responsabilizar-se junto com a empresa pelo cumprimento das disposições legais e regulamentares, incluindo os procedimentos internos de segurança e saúde.
> * Comunicar imediatamente ao responsável pela execução do serviço qualquer situação que considerem de risco para sua segurança e saúde ou para a segurança e saúde de outras pessoas.
> * Exercer o direito de recusa sempre que constatarem evidências de riscos graves e iminentes para sua segurança e saúde ou para a segurança e saúde de outras pessoas.

In [47]:
query = "O que diz o tópico 10.2.9.1 da Norma NR 10?"
to_markdown(rag_chain.invoke(query))

Contexto fornecido: {'context': 'NR 10 - SEGURANÇA EM INSTALAÇÕES E SERVIÇOS EM ELETRICIDADE\n\nPublicação D.O.U. 06/07/78 Alterações/Atualizações D.O.U. 14/06/83 08/09/04 02/05/16 31/07/19\n\nRr - Raio de Faixa delimitação tensão entre zona Nominal da de risco e instalação controlada elétrica em em metros kV 0,20 1 0,22 1 e 3 0,25 3 e 6 0,35 6 e 10 0,38 10 e 15 0,40 15 e 20 0,56 20 e 30 0,58 30 e 36 0,63 36 e 45 0,83 45 e 60 0,90 60 e 70 1,00 70 e 110 110 e 132 1,10 132 e 150 1,20 150 e 220 1,60 220 e 275 1,80 275 e 380 2,50 380 e 480 3,20 480 e 700 5,20 Rc - Raio de delimitação entre zona controlada e livre em metros 0,70 1,22 1,25 1,35 1,38 1,40 1,56 1,58 1,63 1,83 1,90 2,00 3,10 3,20 3,60 3,80 4,50 5,20 7,20\n\n10.8.5 A empresa deve estabelecer sistema de identificação que permita a qualquer tempo conhecer a abrangência da autorização de cada trabalhador, conforme o item 10.8.4.\n\n7\n\n10.8.6 Os trabalhadores autorizados a trabalhar em instalações elétricas devem ter essa condição

> O contexto fornecido não contém informações sobre o tópico 10.2.9.1 da Norma NR 10.

In [49]:
query = "Quais são as exigências da NR 10 para as instalações elétricas em áreas classificadas?"
to_markdown(rag_chain.invoke(query))

Contexto fornecido: {'context': 'NR 10 - SEGURANÇA EM INSTALAÇÕES E SERVIÇOS EM ELETRICIDADE\n\n10.8.5 A empresa deve estabelecer sistema de identificação que permita a qualquer tempo conhecer a abrangência da autorização de cada trabalhador, conforme o item 10.8.4.\n\n7\n\n10.8.6 Os trabalhadores autorizados a trabalhar em instalações elétricas devem ter essa condição consignada no sistema de registro de empregado da empresa.\n\n10.8.7 Os trabalhadores autorizados a intervir em instalações elétricas devem ser submetidos a exame de saúde compatível com as atividades a serem desenvolvidas, realizado em conformidade com a NR 7 e registrado em seu prontuário médico.\n\n10.8.8 Os trabalhadores autorizados a intervir em instalações elétricas devem possuir treinamento específico sobre os riscos decorrentes do emprego da energia elétrica e as principais medidas de prevenção de acidentes em instalações elétricas, de acordo com o estabelecido no Anexo III desta NR. (Alterado pela Portaria MTPS 

> O texto fornecido não contém informações sobre as exigências da NR 10 para instalações elétricas em áreas classificadas.